In [ ]:
import numpy as np
import theano
import theano.tensor as T

In [ ]:
from blocks.bricks import MLP
from blocks.bricks import Rectifier, Logistic
from blocks.bricks.cost import SquaredError
from blocks.graph import ComputationGraph
from blocks.algorithms import GradientDescent, RMSProp
from blocks.initialization import Uniform, Constant
from blocks.extensions.monitoring import DataStreamMonitoring
from blocks.main_loop import MainLoop
from blocks.extensions import FinishAfter, Printing

from fuel.streams import DataStream
from fuel.schemes import SequentialScheme
from fuel.transformers import Flatten

In [ ]:
n_inpt = 25
n_hidden = 250
n_output = 64*64*3
mlp = MLP(activations=[Rectifier(), Logistic()],
         dims=[n_inpt, n_hidden, n_output],
         biases_init=Constant(0))

In [ ]:
mlp.push_initialization_config()
mlp.children[0].weights_init = Uniform(
    width=8.*np.sqrt(6./(n_inpt+n_hidden))/255.)
mlp.children[2].weights_init = Uniform(
    width=8.*np.sqrt(6./(n_inpt+n_hidden)))

mlp.initialize()

In [ ]:
x = T.matrix('label_features', dtype='floatX')
y_hat = mlp.apply(x)
y_raw = T.matrix('image_features', dtype='uint8')
y = T.cast(y_raw, dtype='floatX') / 255.

In [ ]:
cost = SquaredError().apply(y, y_hat)
normalized_cost = cost / (64. * 64. * 3.)
cg = ComputationGraph(cost)

In [ ]:
from fuel.datasets.hdf5 import H5PYDataset
train_set = H5PYDataset('factory_dataset.hdf5', 
                        'train', 
                        subset=slice(0,5000))

test_set = H5PYDataset('factory_dataset.hdf5', 
                       'test')

data_stream_train = Flatten(DataStream.default_stream(
    train_set,
    iteration_scheme=SequentialScheme(train_set.num_examples, 
                                      batch_size=32)))

data_stream_monitor_test = Flatten(DataStream.default_stream(
    test_set,
    iteration_scheme=SequentialScheme(test_set.num_examples, 
                                      batch_size=1024)))

data_stream_monitor_train = Flatten(DataStream.default_stream(
    train_set,
    iteration_scheme=SequentialScheme(train_set.num_examples, 
                                      batch_size=1024)))

In [ ]:
algorithm = GradientDescent(cost=cost, parameters=cg.parameters,
                             step_rule=RMSProp(learning_rate=0.1, 
                                              decay_rate=0.99))

In [ ]:
monitor = DataStreamMonitoring(
    variables=[cost], data_stream=data_stream_monitor_test, prefix="test")
monitor2 = DataStreamMonitoring(
    variables=[cost], data_stream=data_stream_monitor_train, prefix="train")


In [ ]:
main_loop = MainLoop(data_stream=data_stream_train, 
                     algorithm=algorithm,
                     extensions=[monitor, 
                                 monitor2,
                                 FinishAfter(after_n_epochs=5), 
                                 Printing()])

In [ ]:
main_loop.run() 

In [ ]:
epoch = data_stream.get_epoch_iterator()

In [ ]:
ex_image, ex_label = next(epoch)

cost.eval({x: ex_label, y_raw: ex_image})